In [47]:
import json, os
import pandas as pd
from tqdm.auto import tqdm
from neo4j import GraphDatabase
import numpy as np
import re

In [48]:
with open(r"D:\BrainyBlaze\BB_Repos\access\neo4j.json", 'r') as f:
    neo4j_creds = json.load(f)

if 0:   # cloud base
    pass
    # URI = neo4j_creds['url']
    # AUTH = (neo4j_creds['user'], neo4j_creds['password'])
    # Database=neo4j_creds['database']
else:  # local
    URI = "bolt://localhost:7687"
    AUTH = ('neo4j', neo4j_creds['password'])
    Database='mm-lessons'  # manual ravgptv4 neo4j

VECTOR_INDEX_NAME = "lessons_text_embd"
# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH, max_connection_pool_size=25)
driver.verify_connectivity()
print("Connection established.")

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


Connection established.


In [49]:
import openai

In [50]:
with open(r"D:\Programming\My_reps\OpenAI_API_my.txt", 'r') as f:
    openai_key = f.readlines()[0]
client = openai.OpenAI(api_key=openai_key)

def get_embedding(text, model='text-embedding-3-small'):
    if isinstance(text, str):
        text = [text]
    out = client.embeddings.create(input = text, model=model)
    return [o.embedding for o in out.data]

In [55]:
query_sim = """CALL db.index.vector.queryNodes($vector_index_name, 30, $query_vector) YIELD node, score 
RETURN node.lesson_name AS lesson_name, node.time_start AS start, node.time_end AS end, node.upload_date AS upload_date, 
node.youtube_id AS youtube_id, round(score * 1000) / 1000 AS search_score LIMIT $top_k
"""

In [52]:
q = "Что такое цдака?"

embd = get_embedding(q)[0]

In [56]:
result, _, _ = driver.execute_query(query_sim, 
                                    vector_index_name=VECTOR_INDEX_NAME, top_k=10, query_vector=embd,
                                    database_=Database)

In [57]:
result

[<Record lesson_name='Рав Яаков Меир Регев. Мильхама (26)  Мужество Израиля (9). Гоель Цедек(1).' start=1077.64 end=1200.26 upload_date='20-03-2024' youtube_id='_gCiyScrdB0' search_score=0.713>,
 <Record lesson_name='Рав Элиягу Бережинский. Мегилат Рут (2).' start=3599.72 end=3710.66 upload_date='19-05-2024' youtube_id='91MspyQN5kY' search_score=0.703>,
 <Record lesson_name='Рав Йосеф Менделевич. Я и Сталин.' start=1679.52 end=1794.78 upload_date='03-10-2023' youtube_id='a5PoIPzhaIY' search_score=0.697>,
 <Record lesson_name='Рав Яаков Меир Регев  Месилат Йешарим, 13 глава .(1).' start=713.74 end=840.24 upload_date='22-10-2023' youtube_id='lq8ZJPYrqKE' search_score=0.694>,
 <Record lesson_name='Рав Яаков Меир Регев. Месилат Йешарим, 13 глава (4).' start=1555.96 end=1674.52 upload_date='13-11-2023' youtube_id='tzwbbEVzf4Y' search_score=0.692>,
 <Record lesson_name='Рав Йосеф Менделевич. Талмуд по-русски. ＂Бог справедлив？＂' start=2158.8 end=2279.24 upload_date='07-02-2024' youtube_i

In [16]:
result

[<Record lesson_name='Жизнь евреев в древнем Израиле. Урок 14. Талмуд по-русски. Рав Йосеф Менделевич' start=1810.0 end=1919.14 upload_date='07-08-2024' youtube_id='XlE2IuD3Qrg' search_score=0.667>,
 <Record lesson_name='Доносчиков надо убивать. Уроки 9 Ава. Трактат Гитин 55-56. Рав Йосеф Менделевич' start=1559.9 end=1674.5 upload_date='11-08-2024' youtube_id='LJ5gVEQP0Ts' search_score=0.665>,
 <Record lesson_name='Билам-антрополог с психологическим уклоном. Недельная глава Балак. Рав Йосеф Менделевич' start=2032.76 end=2159.3 upload_date='18-07-2024' youtube_id='60NVT2H5LUc' search_score=0.653>,
 <Record lesson_name='Главный закон для еврея. Урок 13. Талмуд по-русски. Рав Йосеф Менделевич' start=2624.64 end=2750.58 upload_date='06-08-2024' youtube_id='93F6lq874K0' search_score=0.644>,
 <Record lesson_name='А разве современные евреи — настоящие？ Недельная глава Пинхас. Рав Йосеф Менделевич' start=4076.18 end=4153.72 upload_date='24-07-2024' youtube_id='PReNV2QCGio' search_score=0.633>